In [2]:
!pip install --upgrade pip

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Requirement already up-to-date: pip in /usr/local/lib/python2.7/dist-packages (20.3.4)


In [3]:
!pip install numpy scipy pandas scikit-learn matplotlib seaborn wordcloud

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


In [4]:
!pip install tensorflow

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |################################| 421.8 MB 6.4 kB/s  eta 0:00:01
     |################################| 24.8 MB 53.9 MB/s eta 0:00:01
     |################################| 3.8 MB 55.6 MB/s eta 0:00:01
     |################################| 42 kB 1.8 MB/s  eta 0:00:01
     |################################| 59 kB 9.9 MB/s  eta 0:00:01
     |################################| 17.0 MB 50.2 MB/s eta 0:00:01
     |################################| 112 kB 76.2 MB/s eta 0:00:01
     |################################| 289 kB 74.3 MB/s eta 0:00:01
     |################################| 4.0 MB 59.5 MB/s et

  Attempting uninstall: numpy
    Found existing installation: numpy 1.15.0
    Uninstalling numpy-1.15.0:
      Successfully uninstalled numpy-1.15.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0
  Attempting uninstall: six
    Found existing installation: six 1.11.0
    Uninstalling six-1.11.0:
      Successfully uninstalled six-1.11.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 40.0.0
    Uninstalling setuptools-40.0.0:
      Successfully uninstalled setuptools-40.0.0
  Attempting uninstall: requests
    Found existing installation: requests 2.19.1
    Uninstalling requests-2.19.1:
      Successfully uninstalled requests-2.19.1


In [5]:
!pip install tensorboard

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


In [6]:
mkdir /tmp/bigdl_summaries

In [113]:
import itertools
import re
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import pandas as pd
from wordcloud import WordCloud
import random as rd
import datetime as dt

In [114]:

from bigdl.dataset import news20
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.util.common import Sample
import time

In [115]:
from bigdl.dataset import mnist
from bigdl.util.common import *
from bigdl.util.common import Sample
from bigdl.dataset.transformer import *
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *

In [116]:
init_engine()

In [117]:
batch_size = 128
embedding_dim = 50
sequence_len = 50
max_words = 1000 
training_split = 0.8

In [41]:
def get_mnist(sc, data_type="train", location="/tmp/mnist"):
    """
    Get and normalize the mnist data. We would download it automatically
    if the data doesn't present at the specific location.

    :param sc: SparkContext
    :param data_type: training data or testing data
    :param location: Location storing the mnist
    :return: A RDD of (features: Ndarray, label: Ndarray)
    """
    (images, labels) = mnist.read_data_sets(location, data_type)
    images = sc.parallelize(images)
    labels = sc.parallelize(labels + 1) # Target start from 1 in BigDL
    record = images.zip(labels)
    return record

In [42]:
from bigdl.dataset import mnist

In [43]:
train_data = get_mnist(sc, "train", "")\
    .map(lambda rec_tuple: (normalizer(rec_tuple[0], mnist.TRAIN_MEAN, mnist.TRAIN_STD),
                       rec_tuple[1]))\
    .map(lambda t: Sample.from_ndarray(t[0], t[1]))
test_data = get_mnist(sc, "test", "")\
    .map(lambda rec_tuple: (normalizer(rec_tuple[0], mnist.TEST_MEAN, mnist.TEST_STD),
                       rec_tuple[1]))\
    .map(lambda t: Sample.from_ndarray(t[0], t[1]))

('Extracting', 'train-images-idx3-ubyte.gz')
('Extracting', 'train-labels-idx1-ubyte.gz')
('Extracting', 't10k-images-idx3-ubyte.gz')
('Extracting', 't10k-labels-idx1-ubyte.gz')


In [118]:
model = Sequential()
model.add(Reshape([1, 28, 28]))
model.add(SpatialConvolution(1, 6, 5, 5))
model.add(Tanh())
model.add(SpatialMaxPooling(2, 2, 2, 2))
model.add(Tanh())
model.add(SpatialConvolution(6, 12, 5, 5))
model.add(SpatialMaxPooling(2, 2, 2, 2))
model.add(Reshape([12 * 4 * 4]))
model.add(Linear(12 * 4 * 4, 100))
model.add(Tanh())
model.add(Linear(100, 100))
model.add(LogSoftMax())

creating: createSequential
creating: createReshape
creating: createSpatialConvolution
creating: createTanh
creating: createSpatialMaxPooling
creating: createTanh
creating: createSpatialConvolution
creating: createSpatialMaxPooling
creating: createReshape
creating: createLinear
creating: createTanh
creating: createLinear
creating: createLogSoftMax


In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='softmax'))


In [75]:
optimizer = Optimizer(
    model=model,
    training_rdd=train_data,
    criterion=ClassNLLCriterion(),
    end_trigger=MaxEpoch(3),
    batch_size=batch_size,
    optim_method=Adagrad(learningrate=.002, learningrate_decay=0.001))

NameError: name 'train_data' is not defined

In [46]:
    optimizer.set_validation(
        batch_size=batch_size,
        val_rdd=test_data,
        trigger=EveryEpoch(),
        val_method=[Top1Accuracy()]
    )

creating: createEveryEpoch
creating: createTop1Accuracy


In [47]:
    logdir='/tmp/bigdl_summaries'
    train_summary = TrainSummary(log_dir=logdir, app_name="app_name")
    train_summary.set_summary_trigger("Parameters", SeveralIteration(50))
    val_summary = ValidationSummary(log_dir=logdir, app_name="app_name")
    optimizer.set_train_summary(train_summary)
    optimizer.set_val_summary(val_summary)

creating: createTrainSummary
creating: createSeveralIteration
creating: createValidationSummary


In [26]:
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.util.common import *
from pyspark import SparkContext
import numpy as np
from bigdl.dataset import mnist


In [49]:
startt = time.time()
trained_model = optimizer.optimize()
endt = time.time()
print("Time taken is " + str(endt-startt))

Time taken is 37.7809531689


In [50]:
results = trained_model.evaluate(test_data, 128, [Top1Accuracy()])
for result in results:
    print(result)

creating: createTop1Accuracy
Evaluated result: 0.883899986744, total_num: 10000, method: Top1Accuracy


In [13]:
!pip install s3fs

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |################################| 47 kB 5.8 MB/s  eta 0:00:01
     |################################| 131 kB 21.2 MB/s eta 0:00:01
     |################################| 7.5 MB 39.0 MB/s eta 0:00:01
     |################################| 79 kB 12.7 MB/s eta 0:00:01
     |################################| 153 kB 80.6 MB/s eta 0:00:01
  Created wheel for s3fs: filename=s3fs-0.2.2-py2.py3-none-any.whl size=19738 sha256=50c828a735b775677e5f04c47f92356e7b5ebb57491be9c669225d3d2f1c6de0
  Stored in directory: /root/.cache/pip/wheels/6a/fe/f2/0a4ecc11da0b15c90aeb5592b1b5f52f9f6c743e3e0d977deb
Successfully 

In [119]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()

In [120]:
key2 = 't_data.npy'
bucket2 = 'mybucket5981'

In [121]:
import boto3
key = 't_train.npy'
bucket = 'mybucket5981'

s3_resource = boto3.resource('s3')

In [122]:
import numpy as np

In [325]:
df = np.load(s3.open('{}/{}'.format(bucket2, key2)))

In [326]:
df = df[0:50000]

In [327]:
key4 = 't_target.npy'
df2 = np.load(s3.open('{}/{}'.format(bucket2, key4)))

In [328]:
key5 = 'test_target.npy'
df4 = np.load(s3.open('{}/{}'.format(bucket2, key5)))

In [329]:
df4 = df4[0:50000]
df2 = df2[0:50000]

In [349]:
x_train=[]
for i in df:
    i = i.astype('float32') / 255.
    i2=i.reshape(-1,28)
    x_train.append(i2)
print(len(x_train[0]))
x_train=x_train[0:50000]

28


In [350]:
y_train=[]
for i in df2:
    y_train.append(i)
y_train=y_train[0:50000]

In [351]:
key3 = 'test_data.npy'
df3 = np.load(s3.open('{}/{}'.format(bucket2, key3)))

In [352]:
key4 = 'test_target.npy'

In [353]:
df4 = np.load(s3.open('{}/{}'.format(bucket2, key4)))

In [354]:
x_test=[]
df3 = df3[0:50000]
for i in df3:
    i = i.astype('float32') / 255.
    i2=i.reshape(-1,28)
    x_test.append(i2)
print(len(x_test[0]))
x_test=x_test[0:50000]

28


In [355]:
x_train5=[]
for i in x_train:
    i = np.expand_dims(i, axis=2)
    x_train5.append(i)

In [356]:
x_test5 = []
for i in x_test:
    i = np.expand_dims(i, axis=2)
    x_test5.append(i)


In [357]:
y_test2=[]
df4 = df4[0:50000]
for i in df4:
    y_test2.append(i)
y_test2=y_test2[0:50000]

In [358]:
print(len(x_train5))
print(len(y_train))
print(len(y_test2))
print(len(x_test5))

50000
50000
50000
50000


In [359]:
y_train2 = np.array(y_train)
y_train2 = y_train2+1
y_test2 = np.array(y_test2)
y_test2 = y_test2+1

In [360]:
x_train5=x_train5[20000:30000]
x_test5=x_test5[20000:30000]
y_train2=y_train2[20000:30000]
y_test2=y_test2[20000:30000]

In [368]:
    imagesr = sc.parallelize(x_train5)
    labelsr = sc.parallelize(y_train2) # Target start from 1 in BigDL
    recordr = imagesr.zip(labelsr)

In [369]:
    imagesr2 = sc.parallelize(x_test5)
    labelsr2 = sc.parallelize(y_test2) # Target start from 1 in BigDL
    recordr2 = imagesr2.zip(labelsr2)

In [370]:
trainw = recordr.map(lambda t: Sample.from_ndarray(t[0], t[1]))
testw = recordr2.map(lambda t: Sample.from_ndarray(t[0], t[1]))

In [371]:
batch_size=128

In [372]:
optimizer2 = Optimizer(
    model=model2,
    training_rdd=trainw,
    criterion=ClassNLLCriterion(),
    end_trigger=MaxEpoch(20),
    batch_size=batch_size,
    optim_method=Adagrad(learningrate=.001, learningrate_decay=0.000))

creating: createClassNLLCriterion
creating: createMaxEpoch
creating: createAdagrad
creating: createDistriOptimizer


In [373]:
    optimizer2.set_validation(
        batch_size=batch_size,
        val_rdd=testw,
        trigger=EveryEpoch(),
        val_method=[Top1Accuracy()]
    )

creating: createEveryEpoch
creating: createTop1Accuracy


In [202]:
    logdir='/tmp/bigdl_summaries'
    train_summary = TrainSummary(log_dir=logdir, app_name="app_name")
    train_summary.set_summary_trigger("Parameters", SeveralIteration(50))
    val_summary = ValidationSummary(log_dir=logdir, app_name="app_name")
    optimizer2.set_train_summary(train_summary)
    optimizer2.set_val_summary(val_summary)

creating: createTrainSummary
creating: createSeveralIteration
creating: createValidationSummary


In [367]:
import time
startt = time.time()
trained_model = optimizer2.optimize()
endt = time.time()
print("Time taken is " + str(endt-startt))

Time taken is 645.320491791


In [374]:
results = trained_model.evaluate(testw, 128, [Top1Accuracy()])
for result in results:
    print(result)

creating: createTop1Accuracy
Evaluated result: 0.556500017643, total_num: 10000, method: Top1Accuracy


In [142]:
model2 = Sequential()
model2.add(Reshape([1, 28, 28]))
model2.add(SpatialConvolution(1, 64, 3, 3))
model2.add(ReLU())
model2.add(SpatialMaxPooling(2, 2, 2, 2))
model2.add(SpatialConvolution(64, 256, 3, 3))
model2.add(ReLU())
model2.add(SpatialMaxPooling(2, 2, 2, 2))
model2.add(SpatialConvolution(256, 512, 3, 3))
model2.add(ReLU())
model2.add(SpatialMaxPooling(2, 2, 2, 2))
model2.add(Reshape([512]))
model2.add(Linear(512, 100))
model2.add(LogSoftMax())

creating: createSequential
creating: createReshape
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createReshape
creating: createLinear
creating: createLogSoftMax


In [278]:
model7 = Sequential()
model7.add(Reshape([1, 28, 28]))
model7.add(SpatialConvolution(1, 4, 3, 3))
model7.add(ReLU())
model7.add(SpatialMaxPooling(2, 2, 2, 2))
model7.add(SpatialConvolution(4, 8, 3, 3))
model7.add(ReLU())
model7.add(SpatialMaxPooling(2, 2, 2, 2))
model7.add(SpatialConvolution(8, 32, 3, 3))
model7.add(ReLU())
model7.add(SpatialMaxPooling(2, 2, 2, 2))
model7.add(Reshape([128*3*3]))
model7.add(Linear(1024, 100))
model7.add(LogSoftMax())

creating: createSequential
creating: createReshape
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createReshape
creating: createLinear
creating: createLogSoftMax


In [ ]:
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(512*3*3, 512),
            nn.Linear(512, numclasses)
        )